# Example 2: Do Wealthier Countries Win More Medals?

## Tasks

#### Correlation
* GDP and total medals
* GDP per capita and medals per capita
* Segment countries into income tiers
* Compare medal distribution across tiers

#### Stats
* Pearson Correlation
* Spearman Correlation (rank-based)

#### Visuals
* Scatter plot with trend line
* Boxplots by income tier